In [ ]:
#importamos librerías
import sys
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


In [ ]:
#cargamos los datos
black_data = pd.read_csv("BlackFriday.csv")
black_data.head()

In [ ]:
#visualizamos las variables numéricas de los datos
black_data.describe()
#verificamos los tipos de valor en cada celda
black_data.dtypes

In [ ]:
cat_values = ["Gender", "Age", "Occupation", "City_Category", "Marital_Status", "Product_Category_1", 
              "Product_Category_2", "Product_Category_3"]
for column in cat_values:
    black_data[column] = black_data[column].astype('category')
black_data.dtypes

In [ ]:
#Suma de valores pérdidos en cada columna. Pandas reconoce tanto una celda vacía como el valor NA como NaN
#https://towardsdatascience.com/data-cleaning-with-python-and-pandas-detecting-missing-values-3e9c6ebcf78b
black_data.isnull().sum()

In [ ]:
#verificamos si hay valores pérdidos con otras sintaxis
#lista con valores NA comunes
missing_values = ["n/a", "na", "--", "Na"]
black_data = pd.read_csv("BlackFriday.csv", na_values= missing_values)
black_data.isnull().sum()

In [ ]:
#verificamos la correlación entre variables para verificar si las variables con valores pérdidos tienen alguna 
#correlación alta con otra variables
black_data.corr()

In [ ]:
#verificamos el nro de filas que tienen valores pérdidos en las 2 variables
len(black_data[black_data['Product_Category_2'].isnull() & black_data['Product_Category_2'].isnull()])


In [ ]:
#verificamos la distribución de Product_Category_1 para comparar con los valores pérdidos de Product_Category_2 y
#Product_Category_3
prod_1 = black_data['Product_ID'].groupby(black_data['Product_Category_1']).nunique()
prod_2 = black_data['Product_ID'].groupby(black_data['Product_Category_2']).nunique()
prod_3 = black_data['Product_ID'].groupby(black_data['Product_Category_2']).nunique()
prod_1

bins = np.sort(black_data['Product_Category_1'].unique())
plt.xticks(bins)
plt.title('Product_Category_1')
plt.bar(bins, prod_1)

In [ ]:
prod_2
prod_3

In [ ]:
#sustituimos los valores pérdidos de Product_category_2 y Product_Categori_3 por 0
bf_data = black_data.fillna(0)
bf_data.isnull().sum()

In [ ]:
#Representamos el gráfico de cajas de la variable "Purchase" y vemos que existen valores extremos 
plt.figure(figsize=[30,10])
plt.subplot(231)
cajas = plt.boxplot(x=bf_data['Purchase'], showmeans = True, meanline = True)
plt.title('Purchase Boxplot')
plt.ylabel('Purchase ($)')

In [ ]:
# Almacenamos los valores extremos en una lista
outliers = list(cajas["fliers"][0].get_data()[1])

# Comprobamos la longitud para ver cuántos registros se consideran extremos.
print "Num. extremos", len(outliers)
print "Num. total", len(bf_data)

# Borramos los datos extremos
bf_data = bf_data[~bf_data.Purchase.isin(outliers)]

# Comparamos las longitudes antes y después
print "Num. sin extremos", len(bf_data)

In [ ]:
# (solo informativo)

# Número de valores diferentes por campo
print('Número de valores distintos: \n')
for i in bf_data.columns:
    print i, ':',bf_data[i].nunique()
    
# Tipo de datos de cada campo
print('\nTipo de datos: \n')
print (bf_data.info())

# Matriz de covarianzas 
print('\nMatriz de covarianzas: \n')
bf_data.cov()

# Análisis de datos

## Selección de los grupos de datos que se quieren analizar/comparar (planificación de los análisis a aplicar)

In [ ]:
for column in cat_values:
    bf_data[column] = bf_data[column].astype('category')
bf_data.dtypes

In [ ]:
#análisis estadístico descriptivo
bf_data.describe()

Observamos que la única variable continua es Purchase. Al utilizar el comando describe observamos también User_ID, pero en este caso al ser la identificación del usuario no tiene sentido los resultados que se indican de las medidas estadísticas de tendencia central y de dispersión.

## Comprobación de la normalidad y homogeneidad de la varianza 

In [ ]:
#visualizamos la distribución de la variable Purchase
plt.hist(bf_data['Purchase'])

In [ ]:
#Aplicamos el test shapiro sobre una muestra porque en la primera prueba obtenemos una advertencia de que el resultado 
#de p-value no es preciso para muestras mayores a 5000
stats.shapiro(bf_data['Purchase'].sample(n=5000, random_state=1))

In [ ]:
#Analizamos la normalidad de la variable Purchase sobre las muestras de hombres y mujeres
purch_female = stats.shapiro(bf_data['Purchase'][bf_data['Gender']=='F'].sample(n=4000, random_state =1))
purch_male = stats.shapiro(bf_data['Purchase'][bf_data['Gender']=='M'].sample(n=4000, random_state =1))
if purch_female[1]<0.05:
    print ("la variable no es normal")
else:
    print ("la variable es normal")

In [ ]:
#Analizamos la normalidad de la variable Purchase sobre las muestras de las variables Gender, Age, 
#City_category y Marital_Status

bf_data['Marital_Status'] = bf_data['Marital_Status'].astype('string')
data_select = ['Gender', 'Age', 'City_Category', 'Marital_Status']
for value in data_select:
    var = bf_data[value].unique()
    for i in range(len(var)):
        shap_num = stats.shapiro(bf_data['Purchase'][bf_data[value]==var[i]].sample(n=4000, random_state =1))
        shap_dict = {value +  var[i]: shap_num[1]}
        print shap_dict
        

In [ ]:
#Probamos la homosteceidad 
for value in data_select:
    var = bf_data[value].unique()
    for i in range(len(var)):
        flig_num = stats.fligner(bf_data['Purchase'], bf_data['Purchase'][bf_data[value]==var[i]])
        if flig_num[1] <0.05:
            flig_stat = u"presentan varianzas diferentes"
        elif flig_num[1] >= 0.05:
            flig_stat = u"presentan varianzas similares"
        flig_dict = {value +  var[i] + " & Purchase": flig_stat.encode('utf8')}
        print flig_dict
        

In [ ]:
# Estudio de la dependencia de variables categóricas: Test Chi Cuadrado 
user_att = ['Gender', 'Age', 'City_Category', 'Marital_Status', 'Occupation', 'City_Category' ]
p_cat = ['Product_Category_1', 'Product_Category_2', 'Product_Category_3']

# Nuestra hipótesis nula será que no existe relación entre la categoría de producto
# y cada una de las variables en data_select.
for i in user_att:
    for j in p_cat:
        tabla_cont = pd.crosstab(bf_data[i], bf_data[j])
        chi2, p, dof, ex = stats.chi2_contingency(tabla_cont, correction=True, lambda_=None)
        if p <0.05:
            print i, ' es independiente de ', j
        else:
            print i, ' puede que sea independiente de Product_Category_1, estudiar más a fondo ', j

In [ ]:
#removemos las variables relacionadas con Product category
bf_select = bf_data.drop(['Product_Category_2', 'Product_Category_3'], axis =1)


In [ ]:
X = bf_select.drop(['Purchase'], axis=1).sample(n=50000, random_state =1)
Y = bf_select['Purchase'].sample(n=50000, random_state =1)
X.head()

In [25]:
level_var = ['Gender', 'Age', 'City_Category', 'Stay_In_Current_City_Years']
#transformamos las variables categóricas en 0s y 1s
for feature in level_var:
    X.loc[:,feature] = LabelEncoder().fit_transform(X[feature])

X.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1
357216,1001068,P00302742,0,1,4,1,2,0,8
389881,1006001,P00354642,0,2,7,0,0,1,8
34323,1005302,P00230842,0,0,10,0,1,0,8
46172,1001121,P00347242,1,6,7,2,4,1,8
379025,1004309,P00042842,1,2,0,2,3,0,5


In [26]:
#separamos las variables con más de 2 categorías

X = pd.DataFrame(OneHotEncoder().fit(X[level_var]).transform(X[level_var]).toarray(), index=X.index, 
                               columns=OneHotEncoder().fit(X[level_var]).get_feature_names(level_var))
X = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns, index=X.index)
X.head()

/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


,Gender_0.0,Gender_1.0,Age_0.0,Age_1.0,Age_2.0,Age_3.0,Age_4.0,Age_5.0,Age_6.0,City_Category_0.0,City_Category_1.0,City_Category_2.0,Stay_In_Current_City_Years_0.0,Stay_In_Current_City_Years_1.0,Stay_In_Current_City_Years_2.0,Stay_In_Current_City_Years_3.0,Stay_In_Current_City_Years_4.0
357216,1.754130,-1.754130,-0.168662,2.115054,-0.808863,-0.504309,-0.299071,-0.275152,-0.201183,-0.603814,1.170904,-0.671941,-0.395361,-0.737642,2.095293,-0.455805,-0.428126
389881,1.754130,-1.754130,-0.168662,-0.472801,1.236303,-0.504309,-0.299071,-0.275152,-0.201183,1.656139,-0.854041,-0.671941,2.529337,-0.737642,-0.477260,-0.455805,-0.428126
34323,1.754130,-1.754130,5.929021,-0.472801,-0.808863,-0.504309,-0.299071,-0.275152,-0.201183,1.656139,-0.854041,-0.671941,-0.395361,1.355672,-0.477260,-0.455805,-0.428126
46172,-0.570083,0.570083,-0.168662,-0.472801,-0.808863,-0.504309,-0.299071,-0.275152,4.970608,-0.603814,-0.854041,1.488225,-0.395361,-0.737642,-0.477260,-0.455805,2.335761
379025,-0.570083,0.570083,-0.168662,-0.472801,1.236303,-0.504309,-0.299071,-0.275152,-0.201183,-0.603814,-0.854041,1.488225,-0.395361,-0.737642,-0.477260,2.193918,-0.428126


In [27]:
#separamos el dataset en train y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

#Aplicamos Random Forest como modelo de clasificación
clf=RandomForestClassifier(n_estimators=10)

clf.fit(X_train,Y_train)

y_pred=clf.predict(X_test)

In [28]:
metrics.accuracy_score(Y_test, y_pred)

0.0002